In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from time import time

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def ChangeDateFormat(date):
    month = date // 100
    day = date % 100
    date = '2020-' + str(month) + '-' + str(day)
    return pd.Timestamp(date)

In [ ]:
# code for importing the original data.
#info = pd.read_csv('/work/ma384/Share/Team_Repeat_Buyer/user_info_format1.csv')
#log = pd.read_csv('/work/ma384/Share/Team_Repeat_Buyer/user_log_format1.csv')
#df_train = pd.read_csv('/work/ma384/Share/Team_Repeat_Buyer/train_format1.csv')


info = pd.read_csv('data_format1/user_info_format1.csv')
log = pd.read_csv('data_format1/user_log_format1.csv')
df_train = pd.read_csv('data_format1/train_format1.csv')

In [ ]:
info['age_range'] = info['age_range'].fillna(0)
info['gender'] = info['gender'].fillna(2)
info['age_range'] = info['age_range'].astype('category')
info['gender'] = info['gender'].astype('category')
info['age_range'] = info['age_range'].replace(8, 7)

In [ ]:
log['time_stamp'] = log['time_stamp'].apply(lambda x: ChangeDateFormat(x))

### Seller infor

In [ ]:
seller_item_count = log[['seller_id','item_id']]
seller_item_count = seller_item_count.drop_duplicates()
seller_item_count['seller_item_count'] = 1
seller_item_count = seller_item_count.groupby(['seller_id']).agg('sum')
seller_item_count = seller_item_count.drop(columns=['item_id'])
seller_item_count.head()

In [2]:
seller_brand_count = log[['seller_id','brand_id']]
seller_brand_count = seller_brand_count.drop_duplicates()
seller_brand_count = seller_brand_count.groupby(['seller_id']).agg(len)
seller_brand_count.rename(columns = {"brand_id": "brand_count"},inplace=True) 
seller_brand_count.head()

NameError: name 'log' is not defined

In [3]:
item_info = log['item_id'].value_counts()
cat_info = log['cat_id'].value_counts()
brand_info = log['brand_id'].value_counts()
seller_info = log['seller_id'].value_counts()
print(item_info.head(), item_info.size)
print(cat_info.head(),cat_info.size)
print(brand_info.head(),brand_info.size)
print(seller_info.head(),seller_info.size)

NameError: name 'log' is not defined

### User action

In [ ]:
actions = log.action_type
actions = pd.get_dummies(actions,prefix='action_count')
user_action = pd.concat([log.user_id,actions],axis=1)
user_action = user_action.groupby('user_id').agg('sum')
user_action.head()

### Merge

In [ ]:
df_Train = pd.read_csv('/work/ma384/Share/Team_Repeat_Buyer/train_format1.csv')

In [ ]:
seller = seller_item_count.merge(seller_brand_count,on='seller_id')
df_train = df_Train.merge(seller,left_on='merchant_id',right_on='seller_id')
df_train = df_train.merge(user_action,on='user_id')
log_dummy = pd.get_dummies(log['action_type'],prefix='action')
log_dummy = pd.concat([log.user_id,log.seller_id,actions],axis=1)
log_dummy.columns = ['user_id', 'merchant_id', 'action_0', 'action_1',
       'action_2', 'action_3']
log_dummy = log_dummy.groupby(['user_id','merchant_id']).agg('sum')
df_train = df_train.merge(log_dummy,on=['merchant_id','user_id'])

In [ ]:
df_train.head(3)

In [ ]:
train, test = train_test_split(df_train,test_size=0.2,random_state=0,shuffle=True,stratify=df_train.label)

In [ ]:
targets_train = train.label.values
targets_test = test.label.values
attributes_train = train.drop(['merchant_id','user_id','label'],axis=1).values
attributes_test = test.drop(['merchant_id','user_id','label'],axis=1).values

In [ ]:
attributes_train = (attributes_train - attributes_train.mean(axis=0))/attributes_train.std(axis=0)
attributes_test = (attributes_test - attributes_test.mean(axis=0))/attributes_test.std(axis=0)

### NN (Stacked LR with ReLU)

In [ ]:
stacked_lr_with_relu = Sequential()
stacked_lr_with_relu.add(Dense(10, input_shape=(10,), activation='relu'))
stacked_lr_with_relu.add(Dense(2, input_shape=(10,), activation=None))

stacked_lr_with_relu.compile(loss='sparse_categorical_crossentropy',
                            optimizer=Adam(learning_rate=1e-5),
                            metrics=['accuracy'])

stacked_lr_with_relu.summary()

In [ ]:
save_best = ModelCheckpoint('nn1.h5', save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
time_start = time()
hist = stacked_lr_with_relu.fit(attributes_train, targets_train, epochs=50, 
                                callbacks=[save_best, early_stopping], 
                                validation_split=0.2, verbose=1)
time_stop = time()
time_elapsed = time_stop - time_start
print(time_elapsed / 60, '(min)')

In [ ]:
targeth_train = np.argmax(stacked_lr_with_relu.predict(attributes_train), axis=1)
targeth_test  = np.argmax(stacked_lr_with_relu.predict(attributes_test), axis=1)

In [ ]:
print("train accuracy", round((targeth_train == targets_train).mean(), 3))
print("test accuracy", round((targeth_test == targets_test).mean(), 3))